In [ ]:
import os
import numpy as np
import sagemaker
from sagemaker.session import s3_input

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

In [ ]:
hyperparams={'epochs': 2,
                 'learning-rate': 0.01,
                 'batch-size': 256,
                 'weight-decay': 2e-4,
                 'momentum': 0.9,
                 'optimizer': 'sgd'}

In [ ]:
from sagemaker.tensorflow import TensorFlow

output_path = 's3://sagemaker-container-webinar/'
tf_estimator = TensorFlow(entry_point='cifar10-training-script-sagemaker.py', 
                          source_dir='code',
                          role=role,
                          train_instance_count=1, 
                          train_instance_type='local_gpu',
                          framework_version='1.13', 
                          py_version='py3',
                          script_mode=True,
                          output_path=output_path,
                          hyperparameters=hyperparams)

#train_instance_type = 'ml.p3.2xlarge'

In [ ]:
%%time
train_path = 's3://sagemaker-container-webinar/cifar10-dataset/train'
val_path = 's3://sagemaker-container-webinar/cifar10-dataset/validation'
eval_path = 's3://sagemaker-container-webinar/cifar10-dataset/eval'

tf_estimator.fit({'training': train_path,'validation': val_path,'eval': eval_path})

In [ ]:
%%time
predictor = tf_estimator.deploy(initial_instance_count=1, instance_type='local_gpu')
#ml.c5.large

In [ ]:
from IPython.display import Image, display

labels = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']
images = []
for entry in os.scandir('sample-img'):
    if entry.is_file() and entry.name.endswith("png"):
        images.append('sample-img/' + entry.name)

for image in images:
    display(Image(image))
    print(image)

In [ ]:
predictor.content_type = 'application/x-image'
predictor.serializer = None

labels = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']

def get_prediction(file_path):
    
    with open(file_path, "rb") as image:
        f = image.read()
    b = bytearray(f)
    return labels[np.argmax(predictor.predict(b)['predictions'], axis=1)[0]]


In [ ]:
predictions = [get_prediction(image) for image in images]

print('Predictions','\t\t\t','Actual')
for pred, actual in zip(predictions,images):
    print(pred,'\t\t\t',actual)

In [ ]:
tf_estimator.delete_endpoint()

In [ ]:
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

hyperparameter_ranges = {
    'epochs':        IntegerParameter(15, 30),
    'learning-rate': ContinuousParameter(0.001, 0.1, scaling_type='Logarithmic'), 
    'batch-size':    IntegerParameter(32, 512),
    'momentum':      ContinuousParameter(0.9, 0.99),
    'optimizer':       CategoricalParameter('sgd', 'adam')
}

objective_metric_name = 'val_acc'
objective_type = 'Maximize'
metric_definitions = [{'Name': 'val_acc', 'Regex': 'val_acc: ([0-9\\.]+)'}]

tuner = HyperparameterTuner(tf_estimator,
                            objective_metric_name,
                            hyperparameter_ranges,
                            metric_definitions,
                            max_jobs=16,
                            max_parallel_jobs=16,
                            objective_type=objective_type)

In [ ]:
tuner.fit({'training': training_input_path, 'validation': validation_input_path})